IMPORTAÇÃO DE MODULOS

In [16]:
from path_arquivos import *
import pandas as pd
import numpy as np
import glob
import os

cons28 = pd.read_excel(bi_abst.cons_28)
cons64 = pd.read_excel(bi_abst.cons_64)
func = pd.read_excel(bi_abst.func, sheet_name='FUNC')
m_atual28 = pd.read_excel(bi_abst.m_atual28)
m_atual64 = pd.read_excel(bi_abst.m_atual64)



c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


EXTRAÇÃO DAS BASES

In [17]:
def organizar_df(df_original, col, id):
    df = df_original
    df[col] = pd.to_datetime(df[col]).dt.normalize()
    df['MES'] = df[col].dt.month

    if id == 1:
        df = df[['NUMOS', 'DATA', 'CODROTINA', 'POSICAO', 'CODFUNCGER', 'FUNCGER','DTFIMOS', 'CODFUNCOS', 'FUNCOSFIM', 'Tipo O.S.', 'TIPOABAST', 'MES']].copy()
        
        df = df.loc[(df['CODROTINA'] == 1709) | (df['CODROTINA'] == 1723)]
        return df
    elif id ==2: 
        df = df[['DATAGERACAO', 'DTLANC', 'NUMBONUS', 'NUMOS', 'CODEMPILHADOR','EMPILHADOR', 'MES']].copy()
        df = df.drop_duplicates(subset=['NUMOS'])
        return df

In [18]:

func = func.loc[func['AREA'] == 'EXPEDICAO']

m_atual28 = organizar_df(m_atual28,'DATA',1)
cons28 = organizar_df(cons28,'DATA',1)
cons28 = cons28.loc[cons28['MES'] != 10]
cons28 = pd.concat([cons28, m_atual28], ignore_index= True)

m_atual64 = organizar_df(m_atual64, 'DATAGERACAO', 2)
cons64 = organizar_df(cons64, 'DATAGERACAO', 2)
cons64 = cons64.loc[cons64['MES'] != 10]
cons64 = pd.concat([cons64, m_atual64], ignore_index= True)

grop28 = cons28.groupby(['DATA','CODFUNCOS','FUNCOSFIM']).agg(
    QTDE_OS = ("NUMOS", 'nunique'),
    OS_50 = ("Tipo O.S.", lambda x: (x == '50 - Movimentação De Para').sum()),
    OS_61 = ("Tipo O.S.", lambda x: (x == '61 - Movimentação De Para Horizontal').sum()),
    OS_58 = ("Tipo O.S.", lambda x: (x == '58 - Transferencia de Para Vertical').sum())
).reset_index()
grop28['50_61'] = grop28['OS_50'] + grop28['OS_61']
grop64 = cons64.groupby(['DATAGERACAO','CODEMPILHADOR']).agg(
    OS_RECEB = ("NUMOS", 'nunique')
).reset_index()

In [19]:
df_os_geral = func.merge(grop28, left_on='ID_FUNC', right_on='CODFUNCOS', how= 'left')
df_os_geral = df_os_geral.merge(grop64, left_on=['ID_FUNC', 'DATA'], right_on= ['CODEMPILHADOR','DATAGERACAO'], how='left').drop(columns=['DATAGERACAO','CODEMPILHADOR', 'CODFUNCOS', 'FUNCOSFIM'])
df_os_geral = df_os_geral.fillna(0)
df_os_geral = df_os_geral.loc[df_os_geral['DATA'] != 0]



bonus = cons64.groupby(['DTLANC']).agg(
    TOTAL_BONUS = ("NUMBONUS", "nunique")
).reset_index().sort_values(by='DTLANC', ascending= False)

In [22]:
os_temp28 = cons28[['NUMOS', 'DATA']].copy()
os_temp64 = cons64[['NUMOS','DATAGERACAO']].copy()
os_temp64 = os_temp64.rename(columns={'DATAGERACAO' : 'DATA'})
os_geral = pd.concat([os_temp28,os_temp64], ignore_index= True)
os_geral = os_geral.groupby('DATA').agg(
    TOTAL_OS = ('NUMOS', 'nunique')
).reset_index()

ped_28 = cons28.loc[cons28['POSICAO'] == "P"]
ped_28 = ped_28.groupby(['CODFUNCGER','DATA']).agg(
    TOTAL_PD = ('NUMOS', 'nunique')
).reset_index()

cons64['CODEMPILHADOR'] = cons64['CODEMPILHADOR'].fillna(0)
cons64['CODFUNCGER'] =1
ped_64 = cons64.loc[cons64['CODEMPILHADOR'] == 0]
ped_64 = ped_64.rename(columns= {'DTLANC':'DATA'})
ped_64 = ped_64.groupby(['CODFUNCGER','DATA']).agg(
    TOTAL_PD = ('NUMOS', 'nunique')
).reset_index()

pd_geral = pd.concat([ped_28,ped_64], ignore_index= True)

In [23]:

display(pd_geral.columns)
display(pd_geral)





Index(['CODFUNCGER', 'DATA', 'TOTAL_PD'], dtype='object')

,CODFUNCGER,DATA,TOTAL_PD
0,1260.0,2025-08-23,1
1,1260.0,2025-10-25,7
2,1260.0,2025-10-27,6
3,1272.0,2025-08-27,2
4,1272.0,2025-10-14,5
...,...,...,...
175,1.0,2025-10-21,14
176,1.0,2025-10-22,13
177,1.0,2025-10-23,26
178,1.0,2025-10-24,42
